In [25]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.llms import OpenAI
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [26]:
#MODEL = "gpt-4-1106-preview"
MODEL = "gpt-3.5-turbo-instruct"

OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"
S2W = "../data/s2w_sample_improved.csv"
S2W_SUMMARIZED = "../data/s2w_summarized.csv"
S2W_SAMPLE = "../data/s2w_sample.csv"

In [27]:
s2w = pd.read_csv(S2W)
s2w.drop(columns=["Unnamed: 0"], inplace=True)

s2w_summarized = pd.read_csv(S2W_SUMMARIZED)
s2w_summarized.drop(columns=["Unnamed: 0"], inplace=True)

s2w_sample = pd.read_csv(S2W_SAMPLE)
s2w_sample.drop(columns=["Unnamed: 0"], inplace=True)

In [28]:
from notebooks.prompts import CREATION_PROMPT_FS, CREATION_PROMPT_ZS, CREATION_PROMPT_COT, CREATION_PROMPT_FS_LLM, CREATION_PROMPT_ZS_LLM, CREATION_PROMPT_COT_LLM

def create_GUI(id, prompt_type="", input_type="", temp=1):
    

    summary = s2w_summarized[s2w_summarized["screenId"]==id][input_type].iloc[0]

    if MODEL == "gpt-3.5-turbo-instruct":
        OUTPUT_FOLDER = "../generated_GUIs/gpt-3.5-turbo-instruct/"
        llm = OpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)

    elif MODEL == "gpt-4-1106-preview":
        OUTPUT_FOLDER = "../generated_GUIs/gpt-4-1106-preview/"
        llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    else:
        return "ERROR: Unknown model provided!"


    if prompt_type == "ZS":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_ZS, callbacks=[handler])
    elif prompt_type == "FS":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_FS, callbacks=[handler])
    elif prompt_type == "COT":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_COT, callbacks=[handler])
    elif prompt_type == "COT_LLM":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_COT_LLM, callbacks=[handler])
    elif prompt_type == "ZS_LLM":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_ZS_LLM, callbacks=[handler])
    elif prompt_type == "FS_LLM":
        chain = LLMChain(llm=llm, prompt=CREATION_PROMPT_FS_LLM, callbacks=[handler])
    else:
        return "ERROR: Unknown prompt type provided!"

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    if prompt_type == "ZS":
        output_file = OUTPUT_FOLDER + f"/Zero_shot/{input_type}/{id}.html"
    elif prompt_type == "FS":
        output_file = OUTPUT_FOLDER + f"/Few_shot/{input_type}/{id}.html"
    elif prompt_type == "ZS_LLM":
        output_file = OUTPUT_FOLDER + f"/Zero_shot/llm_summarized/{id}.html"
    elif prompt_type == "FS_LLM":
        output_file = OUTPUT_FOLDER + f"/Few_shot/llm_summarized/{id}.html"
    elif prompt_type == "COT":
        output_file = OUTPUT_FOLDER + f"/COT/{input_type}/{id}.html"
    elif prompt_type == "COT_LLM":
        output_file = OUTPUT_FOLDER + f"/COT/llm_summarized/{id}.html"
    
    try: 
        with open(output_file, 'w') as f:
            f.write(response)
    except Exception as e:
        print(f"{id} Failed")
        print(e)


In [29]:
def creation_pipeline(prompt_type, input_type):
    screens = s2w_sample["screenId"].head(10).values
    for screen_id in screens:
        create_GUI(screen_id, prompt_type=prompt_type, input_type=input_type)
        print(f"{screen_id}: creation done")

In [30]:
creation_pipeline("FS", "s2w_summary")

33: creation done
70: creation done
300: creation done
486: creation done
494: creation done
495: creation done
498: creation done
549 Failed
'charmap' codec can't encode character '\u25b4' in position 2725: character maps to <undefined>
549: creation done
596: creation done
761: creation done
